In [ ]:
#YOLOv8のアノテーションモデルを使って植生記号とその他（寺と神社、墓）を分け,それぞれをぽイントベクターファイルに変換する。
#yolov8を使うので、インストールしておくこと
from ultralytics import YOLO
import cv2
import csv
import numpy as np
from osgeo import gdal
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import os

In [ ]:
#画面上の位置を緯度経度に変換するための用意。
def length_of_pixel(image_path):

    image = cv2.imread(image_path)
    height, width = image.shape[:2]
    
    # ラスターファイルを開く
    dataset = gdal.Open(image_path)

    # ラスターの座標変換
    transform = dataset.GetGeoTransform()

    # ラスターの左上と右下の座標を取得
    min_x = transform[0]
    max_y = transform[3]
    max_x = min_x + transform[1] * dataset.RasterXSize
    min_y = max_y + transform[5] * dataset.RasterYSize

    # 一画素ごとの緯度経度の長さを計算
    kx = (max_x-min_x)/width
    ky = (max_y-min_y)/height

    # データセットを閉じる
    dataset = None
    
    return kx, ky, min_x, max_y

In [ ]:
def split_image_and_predict(image_path, output_csv, tile_size=480, overlap_threshold=0.25):
    # 画像の読み込み（この画像はGeotiffであること）
    image = cv2.imread(image_path)
    height, width = image.shape[:2]
    kx, ky, min_lx, max_ly = length_of_pixel(image_path)
    
    # CSVファイルの初期化
    with open(output_csv, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['X', 'Y', 'cls', 'conf'])

        y = 0
        while y < height:
            x = 0
            while x < width:
                # タイルのサイズを計算
                tile_height = min(tile_size, height - y)
                tile_width = min(tile_size, width - x)

                # オーバーラップの判定
                if tile_width < tile_size * overlap_threshold and x + tile_size < width:
                    tile_width = tile_size
                if tile_height < tile_size * overlap_threshold and y + tile_size < height:
                    tile_height = tile_size

                # タイル画像の抽出
                tile = image[y:y+tile_height, x:x+tile_width]

                # YOLOv8モデルで予測
                results = model(tile)

                # 予測結果を取得
                for result in results:
                    for box in result.boxes:
                        center_x, center_y, w, h = [item.item() for item in box.xywh[0]]
                        cls = box.cls[0].item()
                        conf = box.conf[0].item()

                        # 元の画像の座標に変換
                        X = x + center_x
                        Y = y + center_y
                        #緯度経度に計算
                        Lx = min_lx + (X * kx)
                        Ly = max_ly - (Y * ky)
                        
                        # CSVファイルに書き込み
                        writer.writerow([Lx, Ly, int(cls), float(conf)])

                # 次のタイルへ
                x += tile_size
            y += tile_size


In [ ]:
#import pandas as pd
#import os

def split_csv_by_cls(folder_path, base_name):
    """
    指定されたフォルダ内のCSVファイルを、cls列の値によって分離し、保存する関数
　このプログラム内で、地図記号は0から15のクラスに分かれているが、うち７，８，１２は、寺・神社・墓なので、これだけ別ファイルに入れている。

    Parameters:
    folder_path (str): CSVファイルがあるフォルダのパス
    base_name (str): 元となるファイル名（拡張子なし）

    """
    # 元のCSVファイルパス
    csv_file = os.path.join(folder_path, f'{base_name}.csv')

    # データの読み込み
    df = pd.read_csv(csv_file)

    # cls=7,8,12の行を抽出
    df_selected = df[df['cls'].isin([7, 8, 12])]

    # それ以外の行を抽出
    df_others = df[~df['cls'].isin([7, 8, 12])]

    # 抽出結果をそれぞれ別ファイルに保存。v始まりが植生記号。o始まりが他の3つ
    output_csv_selected = os.path.join(folder_path, f'o_{base_name}.csv')
    output_csv_others = os.path.join(folder_path, f'v_{base_name}.csv')

    # ファイルを保存
    df_selected.to_csv(output_csv_selected, index=False)
    df_others.to_csv(output_csv_others, index=False)


In [ ]:
def csvtoPoint(output_csv,output_shp):
  # CSVファイルを読み込む (X, Y, cls, conf)
  df = pd.read_csv(output_csv)
  # 経度Xと緯度Yを使ってポイントジオメトリを作成
  geometry = [Point(xy) for xy in zip(df['X'], df['Y'])]

  # GeoDataFrameを作成
  gdf = gpd.GeoDataFrame(df, geometry=geometry)

  # CRS (座標参照系) を指定 (例: WGS84)
  gdf.set_crs(epsg=4326, inplace=True)

  # SHPファイルに書き出す
  gdf.to_file(output_shp)


In [ ]:
def csvtoPoint(output_csv,output_shp):
  # CSVファイルを読み込む (X, Y, cls, conf)
  df = pd.read_csv(output_csv)
  # 経度Xと緯度Yを使ってポイントジオメトリを作成
  geometry = [Point(xy) for xy in zip(df['X'], df['Y'])]

  # GeoDataFrameを作成
  gdf = gpd.GeoDataFrame(df, geometry=geometry)

  # CRS (座標参照系) を指定 (例: WGS84)
  gdf.set_crs(epsg=4326, inplace=True)

  # SHPファイルに書き出す
  gdf.to_file(output_shp)


In [ ]:
# YOLOモデルの読み込み
#モデルがある場所を指定する
model = YOLO('./runs/detect/train/weights/best_v.pt')

# 対象フォルダのパス。最初のフォルダーはGoetiff化された外邦図が入っている。
#あとの2つは、手動で作成。2番目がcsvデータ用、3番目がポイントベクター用。
folder_path = 'D:/Gaihozu/Japan/Hokkaido/East/'
folder_path2 = 'D:/YOLO/test_csv/'
folder_path3 = 'D:/YOLO/test_shp/'

# フォルダ内のすべての .tif ファイルを処理
for file_name in os.listdir(folder_path):
    if file_name.endswith('.tif'):
        # 拡張子を除いたファイル名（ベース名）を取得
        base_name = os.path.splitext(file_name)[0]

        # 最初に使用するデータ
        image_path = os.path.join(folder_path, file_name)
        
        # 中間データ (CSVファイル) のパス_o
        output_csv = os.path.join(folder_path2, f'{base_name}.csv')

        # 中間データ (CSVファイル) のパス_o
        output_csv_o = os.path.join(folder_path2, f'o_{base_name}.csv')

        # 最終データ (SHPファイル) のパス_o
        output_shp_o = os.path.join(folder_path3, f'o_{base_name}.shp')
        
         # 中間データ (CSVファイル) のパス
        output_csv_v = os.path.join(folder_path2, f'v_{base_name}.csv')

        # 最終データ (SHPファイル) のパス
        output_shp_v = os.path.join(folder_path3, f'v_{base_name}.shp')       
        
        split_image_and_predict(image_path, output_csv,tile_size=480, overlap_threshold=0.25)
        split_csv_by_cls(folder_path2, base_name)      
        csvtoPoint(output_csv_o,output_shp_o)
        csvtoPoint(output_csv_v,output_shp_v)